In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
from transformers import AutoTokenizer, pipeline, logging
from awq import AutoAWQForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [4]:
model_name_or_path = "TheBloke/Llama-2-70B-AWQ"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)

In [6]:
model = AutoAWQForCausalLM.from_quantized(
    model_name_or_path, 
    fuse_layers=True,
    trust_remote_code=False, 
    safetensors=True
)

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

Fusing layers...: 100%|██████████| 80/80 [00:00<00:00, 164.28it/s]


In [7]:
prompt = "Tell me about AI"
prompt_template=f'''{prompt}

'''

In [8]:
print("\n\n*** Generate:")

tokens = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=512
)

print("Output: ", tokenizer.decode(generation_output[0]))



*** Generate:
Output:  <s> Tell me about AI

## Introduction

In this assignment you are going to tell me about AI. You are going to write a document that explains what AI is, what it can do, and what you think it will be able to do in the future. This is an essay-type document, and you should use the following guidelines:

1. Write in English, preferably in American English.
2. Use proper grammar.
3. Your text should be at least 400 words.
4. You should be able to explain AI to someone who knows nothing about it.
5. You should not just list facts, you should also include your opinion.
6. You can use as many sources as you want, but you should cite them.
7. Your document should have a title, and a list of references.
8. The document should be written in a text editor, such as Notepad, Atom, or Sublime.
9. The document should be saved as a plain text file, such as `my_essay.txt`.

### Questions

You should answer the following questions in your essay:

1. What is AI?
2. What is the hi

In [9]:
# Inference can also be done using transformers' pipeline
from transformers import pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:


AttributeError: 'LlamaAWQForCausalLM' object has no attribute 'config'